# Data Frame Operations – Select Clause and Functions

As part of this session we will be understanding how to select or project data from data frames while applying functions to extract required information

* Getting Started
* String Manipulation Functions
* Using withColumn
* Using selectExpr
* Date Manipulation Functions
* Dropping Columns
* User Defined Functions – Simple

### Getting Started

Before getting into Pre-Defined functions available to process the data, let us make sure we have Data Frames to apply these functions.

* We can apply toDF on Seq to create Data Frame out of a typical collection.
* First, we will create Data Frame by name dual with column dummy and value X
* Also, let us create Data Frame for the orders data set.
* Most of the Data Frame APIs such as select, where, groupBy etc take column names in the form of strings or of col type.
* Functions used in these APIs take column names in the form of col type.
* If we have to add a constant value to the existing values in a column, we need to use lit on top of constant value.
* We can also use $ instead of col.
* We pass column names as strings if

In [ ]:
import org.apache.spark.sql.functions._

In [ ]:
val dual = Seq("X").toDF("dummy")

In [ ]:
dual.printSchema

In [ ]:
dual.select("dummy").show

In [ ]:
dual.select(col("dummy")).show

In [ ]:
dual.select($"dummy").show

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.select("order_id").show

Below operation will look for column by name order_id2 and it will fail as our data frame does not have column with that name

orders.select("order_id" + 2).show

In [ ]:
//This will add 2 to each value in order_id
orders.select(col("order_id") + 2).show

In [ ]:
orders.select(upper($"order_status")).show

In [ ]:
orders.select(lower($"order_status")).show

In [ ]:
orders.select(initcap($"order_status")).show

### String Manipulation Functions

Let us go through some of the important functions using which we can manipulate strings. We should spend enough time to understand how to manipulate strings using available functions.

* Case conversion functions – lower, upper, initcap
* Trim functions – trim, rtrim, ltrim
* Padding functions – lpad, rpad
* Typecasting – we can use Hive function cast as part of selectExpr to change the data type of data to its original type (eg: date might be a string, but we can extract year part and convert into an integer)
* getting length

**Extracting Data**

Let us see how we can extract data from the fields of Data Frame.

* tracting data from fixed length records – substring
* tracting data from variable length records – split

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.select("order_id").show

In [ ]:
orders.select(upper($"order_status")).show

In [ ]:
orders.select(lower($"order_status")).show

In [ ]:
orders.select(initcap($"order_status")).show

In [ ]:
// trimming unnecessary characters
val dual = Seq("          X       ").toDF("dummy")

In [ ]:
dual.select(trim($"dummy")).show

In [ ]:
dual.select(length(trim($"dummy"))).show

In [ ]:
dual.select(trim($"dummy", " ")).show

In [ ]:
val dual = Seq(7).toDF("dummy")
dual.select(lpad($"dummy", 2, "0")).show

In [ ]:
val dual = Seq(10).toDF("dummy")
dual.select(lpad($"dummy", 2, "0")).show

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.select(substring($"order_date", 1, 4)).show

In [ ]:
// Using split
// Read text data using spark.read.text
// Creates Data Frame using single field with name value
val orders = spark.read.text("/public/retail_db/orders")
orders.select(split($"value", ",")(0).alias("order_id"),
              split($"value", ",")(1).alias("order_date"),
              split($"value", ",")(2).alias("order_customer_id"),
              split($"value", ",")(3).alias("order_status")
             ).show

In [ ]:

val orders = spark.read.json("/public/retail_db_json/orders")
orders.select(substring($"order_date", 1, 4)).show

In [ ]:
orders.select(substring($"order_date", 1, 4).cast("int").alias("order_year")).show


In [ ]:
val orders = spark.read.text("/public/retail_db/orders")
orders.select(split($"value", ",")(0).cast("int").alias("order_id"),
              split($"value", ",")(1).alias("order_date"),
              split($"value", ",")(2).cast("int").alias("order_customer_id"),
              split($"value", ",")(3).alias("order_status")
             ).show

### Using withColumn

We can use withColumn to transform data of a particular column within a Data Frame without impacting other columns.

* If we want to select all the columns as well as new columns by applying some transformation logic, instead of specifying all the columns with select as well as expression – we can use withColumn.
* All the other columns will remain untouched.
* Syntax of withColumn – **df.withColumn(‘column_name’, EXPRESSION)**
* In the below example we are going to discard the timestamp from order_date and then giving the column name as order_date. New Data Frame will still have 4 columns but order_date will have dates without timestamps.

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.printSchema
orders.show

In [ ]:
val ordersNew = orders.
  withColumn("order_year", substring($"order_date", 1, 4)).
  withColumn("order_date", split($"order_date", " ")(0))

In [ ]:
orders.withColumn("order_year", expr("cast(substr(order_date, 1, 4) as int)")).show

In [ ]:
orders.withColumn("order_year", substring($"order_date", 1, 4).cast("int")).show

In [ ]:
orders.
  withColumn("order_status", 
    expr("case when order_status in ('COMPLETE', 'CLOSED') " + 
      "then 'COMPLETED' else order_status end")).
  show

In [ ]:
ordersNew.printSchema

In [ ]:
ordersNew.show

### Using selectExpr

selectExpr for advanced transformations like the **case when**.

* Even though functions available on Data Frames are robust, sometimes we might have to use traditional SQL style approach while applying functions.
* We can take care of it using selectExpr. Whatever functions we pass to selectExpr should follow Hive syntax. If we use split, we need to use [] to access elements from the array generated as result for the split.
* We also have expr which works in similar fashion while using withColumn or while using select.
* selectExpr(“EXPRESSION”) is alias for select(expr(“EXPRESSION”))

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.printSchema
orders.show

In [ ]:
orders.selectExpr("cast(substr(order_date, 1, 4) as int) order_year").show

In [ ]:
orders.
  selectExpr("case when order_status in ('COMPLETE', 'CLOSED') " + 
    "then 'COMPLETED' else order_status end").show

In [ ]:
orders.
  selectExpr("case when order_status in ('COMPLETE', 'CLOSED') " + 
    "then 'COMPLETED' else order_status end order_status").
  show

In [ ]:
orders.
  select(expr("case when order_status in ('COMPLETE', 'CLOSED') " + 
    "then 'COMPLETED' else order_status end order_status")).
  show

In [ ]:
orders.
  withColumn("order_status", 
    expr("case when order_status in ('COMPLETE', 'CLOSED') " + 
      "then 'COMPLETED' else order_status end")).
  show

### Date Manipulation Functions

We also have to deal with dates very often. There are several functions which can be leveraged to manipulate dates.

* Data arithmetic – date_add, date_sub, datediff, next_day, last_day, months_between, add_months
* Getting the first of the month or year – trunc
* Extracting information – dayofmonth, dayofyear, dayofweek, year, month
* Formatting date – date_format
* Typecasting – we can use the cast to convert data type of values of a particular column to its original type.

In [ ]:
val dual = Seq(10).toDF("dummy")

dual.select(current_date()).show

In [ ]:
dual.select(current_timestamp()).show

In [ ]:
dual.select(current_timestamp()).first

In [ ]:
dual.select(date_add(current_date, 3)).show

In [ ]:
dual.select(date_add(current_date, -3)).first

In [ ]:
dual.select(date_sub(current_date, 3)).first

In [ ]:
dual.
  select(datediff(lit("2018-07-25"), lit("2018-06-15"))).
  show

In [ ]:
val dual = Seq(10).toDF("dummy")

In [ ]:
dual.select(trunc(current_date, "mm")).first

In [ ]:
dual.select(trunc(current_date, "yy")).first

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.where("order_date > trunc('2014-07-01', 'mm')").show

In [ ]:
orders.where($"order_date" > trunc(lit("2014-07-01"), "mm")).show

In [ ]:
dual.select(current_date()).first

In [ ]:
dual.select(year(current_date())).first

In [ ]:
dual.select(month(current_date())).first

In [ ]:
dual.select(dayofyear(current_date())).first

In [ ]:
orders.
  withColumn("order_month", date_format($"order_date", "YYYYMM")).
  show

In [ ]:
orders.
  withColumn("order_date", date_format($"order_date", "YYYYMMdd").cast("int")).
  show

In [ ]:
orders.
  select(date_format($"order_date", "YYYYMMdd").cast("int")).
  show

### Dropping Columns

Now let us see how we can drop columns from Data Frame.

* We can use **drop** to drop one or more columns from a Data Frame
* When we use **drop**, it will create new Data Frame without the columns specified as part of drop.
* Column Names can be passed as string type or column type. Using column type we can drop only one column at a time.

In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")

orders.drop($"order_id").show
orders.drop("order_id", "order_status").show

### User Defined Functions – Simple

Let us explore how we can define simple user defined functions and use them as part of Data Frame Operations as well as Spark SQL.

* There are simple UDFs as well as Aggregated UDFs. For now, we will focus on simple UDFs.
* We can create variable for function and convert into UDF, using **org.apache.spark.sql.functions.udf**. It will return object of type **org.apache.spark.sql.expressions.UserDefinedFunction**
Once UDF is created we can use it as part of any transformation function such as select, where, filter, groupBy etc by using Data Frame Native approach.
However, if we want to use it as part of SQL style syntax, we need to register UDF as SQL function. We can do so, by using spark.register.udf.
Let us see a demo by creating UDF to extract year from date string and convert it to Integer.


In [ ]:
val orders = spark.read.json("/public/retail_db_json/orders")
orders.show

val getYearAsInt: String => Int = _.split("-")(0).toInt

import org.apache.spark.sql.functions.udf

val getYearAsIntUDF = udf(getYearAsInt)

orders.select(getYearAsIntUDF($"order_date").alias("order_year")).show

In [ ]:
orders.withColumn("order_year", getYearAsIntUDF($"order_date")).show

In [ ]:
orders.filter(getYearAsIntUDF($"order_date") === 2014).show

In [ ]:
// Register as temporary function to use as part of SQL Style Syntax
spark.udf.register("getYearAsIntUDF", getYearAsInt)
orders.where("getYearAsIntUDF(order_date) = 2014").show

In [ ]:
orders.createTempView("orders")
spark.sql("select o.*, getYearAsIntUDF(o.order_date) order_year from orders o").show